<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Literature_match_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd
import seaborn as sns

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [206]:
Auto_5_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_185_TF_corrected.csv')
Auto_4_gp_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_172_TF_corrected.csv')
Auto_5_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_183_TF_corrected.csv', error_bad_lines=False)
Auto_4_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PF_120k_179_TF_corrected.csv')
QC_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/QC_PUF4_2_120k_159_TF_corrected.csv')

<ipython-input-206-878d1d7399cd>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Auto_5_pm_df = pd.read_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_183_TF_corrected.csv', error_bad_lines=False)
Skipping line 121: expected 21 fields, saw 23



In [207]:
Auto_5_gp_df['Ion Type'] = 'gp'
Auto_4_gp_df['Ion Type'] = 'gp'
Auto_5_pm_df['Ion Type'] = 'gp'
Auto_4_pm_df['Ion Type'] = 'gp'
Auto_5_gp_df['Sample Name'] = 'Auto_5'
Auto_4_gp_df['Sample Name'] = 'Auto_4'
Auto_5_pm_df['Sample Name'] = 'Auto_5'
Auto_4_pm_df['Sample Name'] = 'Auto_4'

In [208]:
print(Auto_5_gp_df.columns)

Index(['Component Name', 'Retention Time', 'Reference m/z', 'Area', 'Height',
       'TIC', 'Formula (mol ion)', 'CAS No.', 'SI', 'RSI', 'HRF Score',
       'RHRF Score', 'Total Score', 'Selected Column Type', 'Calculated RI',
       'Library RI', 'Delta RI', 'Library Name', 'Library ID Number',
       'Sample Name', 'File name', 'Ion Type'],
      dtype='object')


In [209]:
print('Auto5_gp:', Auto_5_gp_df.shape)
print('Auto4_gp:', Auto_4_gp_df.shape)
print('Auto5_pm:', Auto_5_pm_df.shape)
print('Auto4_pm:', Auto_4_pm_df.shape)

Auto5_gp: (259, 22)
Auto4_gp: (465, 22)
Auto5_pm: (182, 22)
Auto4_pm: (197, 22)


In [210]:
def area_ratio(df):
    """
    Calculates ratio of area to the internal standard and saves it in a new column
    """
    benzyl_benzoate_area = df.loc[df['Component Name'] == 'Benzyl Benzoate', 'Area'].values[0]
    df['Area Ratio'] = df['Area'] / benzyl_benzoate_area

    return df

def filter_dataframe(df):
    """
    Removing the results below the probability threshold that is set below
    """
    condition1 = df['RHRF Score'] >= 90
    condition2 = df['RSI'] >= 600
    condition3 = (df['Total Score'] >= 70) & ~(condition1 & condition2)

    filtered_df = df[condition1 & condition2 | condition3]

    return filtered_df

def rearrange_by_area_ratio(df):
    """
    Rearrearanging the rows in descending fashion from the peak area ratio
    """
    if 'Area Ratio' not in df.columns:
        raise ValueError("DataFrame must have a 'Area Ratio' column")

    # Sort the DataFrame based on the 'Area Ratio' column in descending order
    df_sorted = df.sort_values(by='Area Ratio', ascending=False)

    # Reset the index to reflect the new order
    df_sorted = df_sorted.reset_index(drop=True)

    return df_sorted

import pandas as pd

def find_common_row(sample_df, ref_df):
    result_df = pd.DataFrame(columns=sample_df.columns)

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df)
        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)
            if best_match is not None and sample_row['Area Ratio'] >= (0.05 * best_match['Area Ratio']):
                result_df = result_df.append(sample_row, ignore_index=True)

    # Ensure that all Component Names in the result_df are present in the DataFrame before returning
    result_df = result_df[result_df['Component Name'].isin(ref_df['Component Name'])]

    return result_df

def qc_correction(sample_df, ref_df):
    result_df = sample_df.copy()
    matched_component_names = set()

    for _, sample_row in sample_df.iterrows():
        matching_rows = find_matching_rows(sample_row, ref_df)

        if not matching_rows.empty:
            best_match = find_best_match(sample_row, matching_rows)

            if best_match is not None:
                # Modify the condition based on your requirements
                if sample_row['Area Ratio'] >= (0.01 * best_match['Area Ratio']):
                    matched_component_names.add(best_match['Component Name'])

    result_df = result_df[~result_df['Component Name'].isin(matched_component_names)]

    return result_df

def find_matching_rows(sample_row, ref_df):
    matching_rows = ref_df[ref_df['Formula (mol ion)'] == sample_row['Formula (mol ion)']]
    return matching_rows

def find_best_match(sample_row, matching_rows):
    best_match = None
    min_diff = float('inf')

    for _, ref_row in matching_rows.iterrows():
        diff = compare_strings(sample_row['Component Name'], ref_row['Component Name'])
        if diff < min_diff:
            min_diff = diff
            best_match = ref_row

    return best_match

def compare_strings(str1, str2):
    # Implement your string comparison logic here
    # For simplicity, let's assume an exact match for now
    return 0 if str1 == str2 else 1


In [211]:
Auto_5_gp_df = area_ratio(Auto_5_gp_df)
Auto_4_gp_df = area_ratio(Auto_4_gp_df)
Auto_5_pm_df = area_ratio(Auto_5_pm_df)
Auto_4_pm_df = area_ratio(Auto_4_pm_df)
QC_df = area_ratio(QC_df)
print(rearrange_by_area_ratio(QC_df))

                                        Component Name  Retention Time  \
0                              2,4-Di-tert-butylphenol          13.724   
1                                      Benzyl Benzoate          18.606   
2                                        Benzothiazole          10.349   
3                             Butylated Hydroxytoluene          13.673   
4                                        7-Tetradecene          11.249   
..                                                 ...             ...   
338   Benzamide, N-(aminocarbonyl)-N-(1-thioxoethoxy)-          15.706   
339                            p-Methoxyheptanophenone           9.764   
340             2H-Pyran, 2-(4-bromobutoxy)tetrahydro-           5.239   
341           (E)-4-Hydroxy-2-methyl-pent-2-enoic acid           6.897   
342  4H-Cyclopenta[b]thiophene-3-carboxylic acid, 2...          24.900   

     Reference m/z        Area     Height        TIC Formula (mol ion)  \
0       191.142929  2209640031  38809

In [212]:
threshold_value = 0.06 # Set your desired threshold value

Auto_5_gp_df_filtered = qc_correction(Auto_5_gp_df, QC_df)
Auto_4_gp_df_filtered = qc_correction(Auto_4_gp_df, QC_df)
Auto_5_pm_df_filtered = qc_correction(Auto_5_pm_df, QC_df)
Auto_4_pm_df_filtered = qc_correction(Auto_4_pm_df, QC_df)
print('Auto5_gp:', Auto_5_gp_df_filtered.shape)
print('Auto4_gp:', Auto_4_gp_df_filtered.shape)
print('Auto5_pm:', Auto_5_pm_df_filtered.shape)
print('Auto4_pm:', Auto_4_pm_df_filtered.shape)
#Auto_5_gp_df_filtered.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_corrected.csv')

Auto5_gp: (195, 23)
Auto4_gp: (362, 23)
Auto5_pm: (136, 23)
Auto4_pm: (147, 23)


In [213]:
Auto5_gp_comon_df = find_common_row(Auto_5_gp_df, QC_df)
Auto4_gp_comon_df = find_common_row(Auto_4_gp_df, QC_df)
Auto5_pm_comon_df = find_common_row(Auto_5_pm_df, QC_df)
Auto4_pm_comon_df = find_common_row(Auto_4_pm_df, QC_df)
print('Auto5_gp:', Auto5_gp_comon_df.shape)
print('Auto4_gp:', Auto4_gp_comon_df.shape)
print('Auto5_pm:', Auto5_pm_comon_df.shape)
print('Auto4_pm:', Auto4_pm_comon_df.shape)
#print(rearrange_by_area_ratio(Auto5_gp_comon_df))
#Auto5_gp_comon_df.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_matches.csv')

<ipython-input-210-30faddb49197>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-210-30faddb49197>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-210-30faddb49197>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)
<ipython-input-210-30faddb49197>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sample_row, ignore_index=True)


Auto5_gp: (64, 23)
Auto4_gp: (102, 23)
Auto5_pm: (45, 23)
Auto4_pm: (50, 23)


In [214]:
Auto5_gp_ordered = rearrange_by_area_ratio(Auto_5_gp_df_filtered)
Auto4_gp_ordered = rearrange_by_area_ratio(Auto_4_gp_df_filtered)
Auto5_pm_ordered = rearrange_by_area_ratio(Auto_5_pm_df_filtered)
Auto4_pm_ordered = rearrange_by_area_ratio(Auto_4_pm_df_filtered)
print(Auto5_gp_ordered.head())
print(Auto4_gp_ordered.head())
print(Auto5_pm_ordered.head())
print(Auto4_pm_ordered.head())

                                      Component Name  Retention Time  \
0  Benzenepropanoic acid, 3,5-bis(1,1-dimethyleth...          33.695   
1                        Bis(2-ethylhexyl) phthalate          26.756   
2            Benzenamine, 4-octyl-N-(4-octylphenyl)-          33.542   
3                       2,6-Octadiene, 2,4-dimethyl-          10.704   
4                           Benzothiazole, 2-methyl-          11.358   

   Reference m/z        Area     Height         TIC Formula (mol ion)  \
0     219.174301  3487699203  201822395  2219112596          C35H62O3   
1     149.023331  1021649525  180457900   266915908          C24H38O4   
2     322.252838   763432606  119074220   250200421           C28H43N   
3      83.085503   519824310   68441127   136144784            C10H18   
4     149.029358   446572840   80723631   195606934            C8H7NS   

      CAS No.   SI  RSI  ...  Selected Column Type  Calculated RI  Library RI  \
0   2082-79-3  828  835  ...                  N

In [215]:
Autp5_gp_final = filter_dataframe(Auto5_gp_comon_df)
Autp4_gp_final = filter_dataframe(Auto4_gp_comon_df)
Autp5_pm_final = filter_dataframe(Auto5_pm_comon_df)
Autp4_pm_final = filter_dataframe(Auto4_pm_comon_df)
print('Auto5_gp:', Autp5_gp_final.shape)
print('Auto4_gp:', Autp4_gp_final.shape)
print('Auto5_pm:', Autp5_pm_final.shape)
print('Auto4_pm:', Autp4_pm_final.shape)
Autp5_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PUF_120k_blank_filtered.csv')
Autp4_gp_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PUF_120k_blank_filtered.csv')
Autp5_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto5_PF_120k_blank_filtered.csv')
Autp4_pm_final.to_csv('/content/drive/My Drive/unknown_screening_data/Auto4_PF_120k_blank_filtered.csv')

Auto5_gp: (62, 23)
Auto4_gp: (101, 23)
Auto5_pm: (43, 23)
Auto4_pm: (46, 23)
